In [7]:
from azure.identity import AzureCliCredential, get_bearer_token_provider
import openai

class AzureOpenAIClient:
    """
    A class to interact with Azure OpenAI API using Azure CLI authentication.
    """
    
    def __init__(self, azure_endpoint_url, api_version="2024-02-15-preview", bearer_token_url='https://cognitiveservices.azure.com/.default'):
        """
        Initialize the Azure OpenAI client.
        
        Args:
            azure_endpoint_url (str): The Azure OpenAI endpoint URL
            api_version (str): The API version to use
            bearer_token_url (str): The bearer token URL for authentication
        """
        self.azure_endpoint_url = azure_endpoint_url
        self.api_version = api_version
        self.bearer_token_url = bearer_token_url
        self.credential = self._get_cli_credential()
        self.token_provider = self._get_token_provider()
        self.client = self._get_openai_client()
    
    def _get_cli_credential(self):
        """Get Azure CLI credentials."""
        return AzureCliCredential()
    
    def _get_token_provider(self):
        """Get bearer token provider."""
        return get_bearer_token_provider(self.credential, self.bearer_token_url)
    
    def _get_openai_client(self):
        """Create and return the OpenAI client."""
        return openai.AzureOpenAI(
            api_version=self.api_version,
            azure_endpoint=self.azure_endpoint_url,
            azure_ad_token_provider=self.token_provider
        )
    
    def get_completion(self, user_prompt, system_prompt=None, model="gpt-4o", max_tokens=100, temperature=0.5, timeout_seconds=10):
        """
        Get a completion from the Azure OpenAI API.
        
        Args:
            user_prompt (str): The user's prompt/question
            system_prompt (str, optional): System prompt to set context/behavior
            model (str): The model to use
            max_tokens (int): Maximum tokens in the response
            temperature (float): Sampling temperature (0-1)
            timeout_seconds (int): Request timeout in seconds
            
        Returns:
            str: The response text from the model
        """
        messages = []
        
        # Add system prompt if provided
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # Add user prompt
        messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = self.client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature,
            timeout=timeout_seconds
        )
        
        # Extract and return the text content
        return response.choices[0].message.content
    
    def get_full_response(self, user_prompt, system_prompt=None, model="gpt-4o", max_tokens=100, temperature=0.5, timeout_seconds=10):
        """
        Get the full response object from the Azure OpenAI API.
        
        Args:
            user_prompt (str): The user's prompt/question
            system_prompt (str, optional): System prompt to set context/behavior
            model (str): The model to use
            max_tokens (int): Maximum tokens in the response
            temperature (float): Sampling temperature (0-1)
            timeout_seconds (int): Request timeout in seconds
            
        Returns:
            ChatCompletion: The full response object
        """
        messages = []
        
        # Add system prompt if provided
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # Add user prompt
        messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = self.client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature,
            timeout=timeout_seconds
        )
        
        return response

# Initialize the client; these are custom values for an Azure OpenAI endpoint accessible to a team member
azure_endpoint_url = 'https://winsightsbotwus.openai.azure.com/'
api_version = "2024-02-15-preview"

ai_client = AzureOpenAIClient(azure_endpoint_url, api_version)

print("Azure OpenAI Client initialized successfully!")

Azure OpenAI Client initialized successfully!


In [3]:
import json
import os

# Get the directory where this notebook is located
notebook_dir = os.path.dirname(os.path.abspath("SampleHabitLoopPlan.ipynb"))

# Load the evaluation results
results_file = os.path.join(notebook_dir, "evaluation_results_20251004_135831.json")

with open(results_file, 'r', encoding='utf-8') as f:
    evaluation_data = json.load(f)

# Display summary
print(f"Loaded evaluation results from: {results_file}")
print(f"\nMetadata:")
print(json.dumps(evaluation_data['metadata'], indent=2))

# Get the results list
evaluation_results = evaluation_data['results']
print(f"\n{'='*50}")
print(f"Number of result entries: {len(evaluation_results)}")
print(f"{'='*50}")

# Display first result
if len(evaluation_results) > 0:
    print("\nFirst result entry:")
    print(json.dumps(evaluation_results[0], indent=2)[:1000])  # First 1000 chars

Loaded evaluation results from: c:\Hackathon\Oct-4-Hackathon-2025-hack16\resources\SampleHabitLoopPlan\evaluation_results_20251004_135831.json

Metadata:
{
  "processed_at": "2025-10-04T13:54:29.042581",
  "system_prompt_file": "SustainabilityCoach-SystemPrompt.txt",
  "model": "gpt-4o",
  "total_entries": 30,
  "total_inputs_processed": 95
}

Number of result entries: 95

First result entry:
{
  "category": "Morning Routine (Home)",
  "demographic": "Young Professional, Female, 28",
  "input_index": 0,
  "original_input": "Hey, I'm about to order an Uber to get to my office downtown",
  "raw_response": "```json\n{\n  \"original_action\": \"Taking an Uber to the office downtown\",\n  \"suggestion\": \"Use public transportation or a bike-sharing service if available in your area.\",\n  \"rationale\": \"Public transportation or biking significantly reduces carbon emissions compared to taking a ride-hailing service. These options are often cost-effective, and biking has the added benefit 

In [4]:
import random

# Set seed for reproducibility
random.seed(42)

# Select 8 random sample responses from the evaluation results
sample_size = 8
selected_indices = random.sample(range(len(evaluation_results)), min(sample_size, len(evaluation_results)))

print(f"Selected {len(selected_indices)} random samples from {len(evaluation_results)} total entries")
print(f"Indices: {selected_indices}\n")

# Extract the responses
sample_responses = []
for idx in selected_indices:
    result = evaluation_results[idx]
    sample_responses.append({
        'index': idx,
        'category': result.get('category', 'N/A'),
        'demographic': result.get('demographic', 'N/A'),
        'original_input': result.get('original_input', ''),
        'parsed_response': result.get('parsed_json', {})
    })

# Display the samples
for i, sample in enumerate(sample_responses, 1):
    print(f"\n{'='*60}")
    print(f"Sample {i} (Index {sample['index']})")
    print(f"{'='*60}")
    print(f"Category: {sample['category']}")
    print(f"Demographic: {sample['demographic']}")
    print(f"User Input: {sample['original_input']}")
    if sample['parsed_response']:
        print(f"\nSuggestion: {sample['parsed_response'].get('suggestion', 'N/A')}")
        print(f"Impact: {sample['parsed_response'].get('impact_indicator', 'N/A')}")

Selected 8 random samples from 95 total entries
Indices: [81, 14, 3, 94, 35, 31, 28, 17]


Sample 1 (Index 81)
Category: Special Situations
Demographic: Moving/Relocating
User Input: Renting a moving truck to move across the state

Suggestion: Choose the smallest truck that fits your belongings efficiently and pack strategically to maximize space usage.
Impact: medium

Sample 2 (Index 14)
Category: Mid-Morning (Work/Errands)
Demographic: Business Executive, Female, 52
User Input: Printing fifty copies of the quarterly report for tomorrow's board meeting

Suggestion: Distribute digital copies of the report via email or a shared platform while printing only a few physical copies for those who explicitly need them.
Impact: high

Sample 3 (Index 3)
Category: Morning Routine (Home)
Demographic: Middle-aged Parent, Male, 45
User Input: I'm driving the kids to school this morning, it's about three miles away

Suggestion: Consider walking or biking to school with the kids if time and weather p

In [5]:
# Compile the 8 samples into an LLM-ready user prompt
user_prompt = """I need help creating a personalized habit loop plan to make my daily actions more sustainable. Below are 8 different situations where I've received sustainability recommendations. Please analyze these and create a comprehensive habit loop plan that helps me build better sustainable habits.

Here are my daily scenarios and the sustainability suggestions I've received:

"""

for i, sample in enumerate(sample_responses, 1):
    parsed = sample['parsed_response']
    user_prompt += f"{i}. {sample['original_input']}\n"
    user_prompt += f"   Recommendation: {parsed.get('suggestion', 'N/A')}\n"
    user_prompt += f"   Why: {parsed.get('rationale', 'N/A')}\n"
    user_prompt += f"   Environmental Impact: {parsed.get('impact_indicator', 'N/A')}\n\n"

user_prompt += """Based on these scenarios, please create a habit loop plan that:
1. Identifies common themes or patterns in my behavior
2. Suggests specific cues or triggers to remind me of sustainable alternatives
3. Defines clear routines/actions I should take in each situation
4. Proposes meaningful rewards to reinforce these new habits
5. Provides a structured approach to gradually adopt these sustainable practices

Please format your response as a comprehensive habit loop plan with actionable steps I can start implementing immediately."""

# Display the compiled prompt
print("="*70)
print("COMPILED USER PROMPT FOR LLM")
print("="*70)
print(user_prompt)
print("\n" + "="*70)
print(f"Total prompt length: {len(user_prompt)} characters")
print("="*70)

COMPILED USER PROMPT FOR LLM
I need help creating a personalized habit loop plan to make my daily actions more sustainable. Below are 8 different situations where I've received sustainability recommendations. Please analyze these and create a comprehensive habit loop plan that helps me build better sustainable habits.

Here are my daily scenarios and the sustainability suggestions I've received:

1. Renting a moving truck to move across the state
   Recommendation: Choose the smallest truck that fits your belongings efficiently and pack strategically to maximize space usage.
   Why: Using a smaller truck reduces fuel consumption and emissions. Packing efficiently ensures fewer trips are needed, lowering the overall environmental impact.
   Environmental Impact: medium

2. Printing fifty copies of the quarterly report for tomorrow's board meeting
   Recommendation: Distribute digital copies of the report via email or a shared platform while printing only a few physical copies for those 

In [6]:
# Store the prompt in a variable for later use with the LLM
habit_loop_user_prompt = user_prompt

# Display summary
print("\n✅ User prompt compiled successfully!")
print(f"\nPrompt Statistics:")
print(f"  - Number of scenarios included: {len(sample_responses)}")
print(f"  - Total characters: {len(user_prompt):,}")
print(f"  - Estimated tokens (rough): ~{len(user_prompt) // 4:,}")
print(f"\nThe prompt is stored in variable: habit_loop_user_prompt")
print(f"\nYou can now use this prompt with the Azure OpenAI client:")
print(f"  response = ai_client.get_completion(habit_loop_user_prompt, max_tokens=2000)")
print("\nFirst 500 characters of the prompt:")
print("-" * 70)
print(user_prompt[:500] + "...")


✅ User prompt compiled successfully!

Prompt Statistics:
  - Number of scenarios included: 8
  - Total characters: 4,574
  - Estimated tokens (rough): ~1,143

The prompt is stored in variable: habit_loop_user_prompt

You can now use this prompt with the Azure OpenAI client:
  response = ai_client.get_completion(habit_loop_user_prompt, max_tokens=2000)

First 500 characters of the prompt:
----------------------------------------------------------------------
I need help creating a personalized habit loop plan to make my daily actions more sustainable. Below are 8 different situations where I've received sustainability recommendations. Please analyze these and create a comprehensive habit loop plan that helps me build better sustainable habits.

Here are my daily scenarios and the sustainability suggestions I've received:

1. Renting a moving truck to move across the state
   Recommendation: Choose the smallest truck that fits your belongings efficie...


In [8]:
# Call the Azure OpenAI API to generate the habit loop plan
print("🔄 Calling Azure OpenAI API to generate habit loop plan...")
print("This may take a moment...\n")

try:
    # Call the API with increased max_tokens for a comprehensive response
    habit_loop_plan = ai_client.get_completion(
        user_prompt=habit_loop_user_prompt,
        model="gpt-4o",
        max_tokens=2500,
        temperature=0.7,
        timeout_seconds=30
    )
    
    print("✅ Successfully received response from Azure OpenAI!")
    print("\n" + "="*70)
    print("PERSONALIZED HABIT LOOP PLAN")
    print("="*70 + "\n")
    print(habit_loop_plan)
    print("\n" + "="*70)
    print(f"Response length: {len(habit_loop_plan)} characters")
    print("="*70)
    
except Exception as e:
    print(f"❌ Error calling Azure OpenAI API: {str(e)}")
    print(f"Error type: {type(e).__name__}")

🔄 Calling Azure OpenAI API to generate habit loop plan...
This may take a moment...

✅ Successfully received response from Azure OpenAI!

PERSONALIZED HABIT LOOP PLAN

### Comprehensive Habit Loop Plan for Building Sustainable Habits

This habit loop plan is designed to help you adopt sustainable practices across the scenarios you encounter daily. It follows the **cue-routine-reward** framework and incorporates gradual, structured steps to ensure long-term habit formation.

---

### **1. Identify Common Themes or Patterns**
After analyzing your scenarios, here are the recurring themes:
- **Resource Efficiency**: Choosing alternatives that use fewer resources (e.g., smaller truck, fewer printed copies).
- **Reducing Waste**: Avoiding single-use items (e.g., reusable cups, water bottles).
- **Energy Conservation**: Optimizing energy use (e.g., server cooling systems, biking instead of driving).
- **Repair Over Replace**: Extending the lifespan of items (e.g., fixing your phone).
- **Proa

In [9]:
# Save the habit loop plan to a file for future reference
from datetime import datetime

# Create a timestamp for the filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = os.path.join(notebook_dir, f"HabitLoopPlan_{timestamp}.txt")

# Save the plan
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("PERSONALIZED HABIT LOOP PLAN\n")
    f.write("="*70 + "\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Based on {len(sample_responses)} sample scenarios\n")
    f.write("="*70 + "\n\n")
    f.write(habit_loop_plan)

print(f"✅ Habit loop plan saved to: {output_file}")
print(f"\nYou can now review and implement the personalized habit loop plan!")

✅ Habit loop plan saved to: c:\Hackathon\Oct-4-Hackathon-2025-hack16\resources\SampleHabitLoopPlan\HabitLoopPlan_20251004_143409.txt

You can now review and implement the personalized habit loop plan!


In [ ]:
from IPython.display import HTML, display
import markdown

# Convert the markdown-formatted plan text to HTML
# Using markdown library to properly convert markdown syntax
plan_html_content = markdown.markdown(
    habit_loop_plan,
    extensions=['extra', 'nl2br', 'sane_lists']
)

# Load the HTML template
template_file = os.path.join(notebook_dir, "habit_loop_template.html")
with open(template_file, 'r', encoding='utf-8') as f:
    html_template = f.read()

# Replace the placeholder with the converted HTML content
html_output = html_template.replace('{{PLAN_CONTENT}}', plan_html_content)

# Display the HTML
display(HTML(html_output))
print("\n✅ HTML rendered with external CSS and converted from markdown!")

In [11]:
# Save the HTML plan to a file
html_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
html_output_file = os.path.join(notebook_dir, f"HabitLoopPlan_{html_timestamp}.html")

# Save the HTML file (with reference to external CSS)
with open(html_output_file, 'w', encoding='utf-8') as f:
    f.write(html_output)

print(f"✅ HTML habit loop plan saved to: {html_output_file}")
print(f"\nFiles generated:")
print(f"  - HTML: {html_output_file}")
print(f"  - CSS: {os.path.join(notebook_dir, 'habit_loop_styles.css')}")
print(f"  - Template: {os.path.join(notebook_dir, 'habit_loop_template.html')}")
print(f"\n💡 Open the HTML file in a web browser to view the formatted plan!")
print(f"   The CSS file must be in the same directory for proper styling.")

✅ HTML habit loop plan saved to: c:\Hackathon\Oct-4-Hackathon-2025-hack16\resources\SampleHabitLoopPlan\HabitLoopPlan_20251004_143554.html

You can open this file in a web browser to view the formatted plan!
File size: 8,948 bytes
